In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
n_hidden_1 = 256
n_hidden_2 = 128
n_input = 784
n_classes = 10

In [4]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [5]:
stddev = 0.1

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [6]:
def multilayer_perceptron(_X, _W, _b):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _W['h1']), _b['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _W['h2']), _b['b2']))
    return tf.matmul(layer_2, _W['out']) + _b['out']    

In [7]:
pred = multilayer_perceptron(x, weights, biases)

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

In [9]:
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [10]:
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

In [11]:
init = tf.global_variables_initializer()

In [12]:
training_epochs = 20
batch_size = 100
display_step = 4

In [13]:
sess = tf.Session()
sess.run(init)

In [14]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))

Epoch: 003/020 cost: 0.118819320
TRAIN ACCURACY: 0.950
TEST ACCURACY: 0.963
Epoch: 007/020 cost: 0.047402968
TRAIN ACCURACY: 0.990
TEST ACCURACY: 0.974
Epoch: 011/020 cost: 0.019407654
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.980
Epoch: 015/020 cost: 0.006871214
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.980
Epoch: 019/020 cost: 0.002370298
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.982
